# Training Vt with Simple MLP Regression 

We found saturation points and use assumption that Vt = Vgs-Vds in saturation point.\
By using that fact, we made Vt datasets.\
Now we'll train model that...\
Vt = F(W,L,T,Vgs,Vds,Ids)

In [1]:
import torch
import pandas as pd
import numpy as np
from torch import nn
from torch.utils.data import DataLoader
from sklearn.preprocessing import StandardScaler

## 1) Make dataset for Vt model

In [2]:
class VtDataset(torch.utils.data.Dataset):
    """
    Vt Dataset
    """
    def __init__(self, X, y, scale_data=False):
        if not torch.is_tensor(X) and not torch.is_tensor(y):
            if scale_data:
                X = StandardScaler().fit_transform(X)
            self.X = torch.from_numpy(X)
            self.y = torch.from_numpy(y)
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, i):
        return self.X[i], self.y[i]

## 2) Make model

In [3]:
class VtMLP(nn.Module):
  '''
    Multilayer Perceptron for regression.
  '''
  def __init__(self):
    super().__init__()
    self.layers = nn.Sequential(
      nn.Linear(5, 64),
      nn.ReLU(),
      nn.Linear(64, 32),
      nn.ReLU(),
      nn.Linear(32, 1)
    )


  def forward(self, x):
    '''
      Forward pass
    '''
    return self.layers(x)

## 3) Training

In [4]:
vt_df = pd.read_csv('../data_vt/vt_dataset.csv')
vt_df.head()

,Vt,W,L,T,Vgs,Vds,Ids
0,-21.0,10,5,0,-20.0,1.0,9.562200e-15
1,-20.8,10,5,0,-19.8,1.0,9.562200e-15
2,-20.6,10,5,0,-19.6,1.0,9.562200e-15
3,-20.4,10,5,0,-19.4,1.0,9.562200e-15
4,-20.2,10,5,0,-19.2,1.0,9.562200e-15


In [5]:
vt_df_X = vt_df[['W', 'L', 'T', 'Vgs', 'Vds']]
vt_X = vt_df_X.to_numpy()
vt_X

array([[  10. ,    5. ,    0. ,  -20. ,    1. ],
       [  10. ,    5. ,    0. ,  -19.8,    1. ],
       [  10. ,    5. ,    0. ,  -19.6,    1. ],
       ...,
       [1000. ,   12. ,   80. ,   19.6,   10. ],
       [1000. ,   12. ,   80. ,   19.8,   10. ],
       [1000. ,   12. ,   80. ,   20. ,   10. ]])

In [6]:
vt_df_y = vt_df[['Vt']]
vt_y = vt_df_y.to_numpy()
vt_y

array([[-21. ],
       [-20.8],
       [-20.6],
       ...,
       [  9.6],
       [  9.8],
       [ 10. ]])

In [7]:
dataset = VtDataset(vt_X, vt_y)
print(len(dataset))
train_loader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True, num_workers=1)

14472


In [8]:
mlp = VtMLP()

In [9]:
loss_function = nn.L1Loss()
optimizer = torch.optim.Adam(mlp.parameters(), lr=1e-4)

In [10]:
# Run the training loop
for epoch in range(0, 20): # 20 epochs at maximum
    
    # Print epoch
    print(f'Starting epoch {epoch+1}')
    
    # Set current loss value
    current_loss = 0.0
    
    # Iterate over the DataLoader for training data
    for i, data in enumerate(train_loader, 0):
        # Get and prepare inputs
        inputs, targets = data
        inputs, targets = inputs.float(), targets.float()
        targets = targets.reshape((targets.shape[0], 1))
        
        # Zero the gradients
        optimizer.zero_grad()
        
        # Perform forward pass
        outputs = mlp(inputs)
        
        # Compute loss
        loss = loss_function(outputs, targets)
        
        # Perform backward pass
        loss.backward()
        
        # Perform optimization
        optimizer.step()
        
        # Print statistics
        current_loss += loss.item()
        if i % 1000 == 0:
            print('Loss after mini-batch %5d: %.3f' %
                (i + 1, current_loss / 20*32))
            current_loss = 0.0

# Process is complete.
print('Training process has finished.')

Starting epoch 1
Loss after mini-batch     1: 18.978
Starting epoch 2
Loss after mini-batch     1: 6.966
Starting epoch 3
Loss after mini-batch     1: 2.993
Starting epoch 4
Loss after mini-batch     1: 2.238
Starting epoch 5
Loss after mini-batch     1: 2.613
Starting epoch 6
Loss after mini-batch     1: 0.986
Starting epoch 7
Loss after mini-batch     1: 1.144
Starting epoch 8
Loss after mini-batch     1: 0.554
Starting epoch 9
Loss after mini-batch     1: 0.354
Starting epoch 10
Loss after mini-batch     1: 0.655
Starting epoch 11
Loss after mini-batch     1: 0.585
Starting epoch 12
Loss after mini-batch     1: 0.414
Starting epoch 13
Loss after mini-batch     1: 0.437
Starting epoch 14
Loss after mini-batch     1: 0.212
Starting epoch 15
Loss after mini-batch     1: 0.352
Starting epoch 16
Loss after mini-batch     1: 0.155
Starting epoch 17
Loss after mini-batch     1: 0.216
Starting epoch 18
Loss after mini-batch     1: 0.192
Starting epoch 19
Loss after mini-batch     1: 0.150
S

## 4) Saving...

In [11]:
torch.save(mlp.state_dict(), '../checkpoint/vt_mlp_model.pt')